## Code load dump file using Ovito library.
### Here you have to set path to .dump file

In [5]:
from ovito.io import import_file
from ovito.modifiers import *
import matplotlib.pyplot as plt
import numpy as np
import os

# --- Configuration --- #
plik_dump = "/home/przemek/Documents/temp/heating_pd_10k_10ps.dump"  # path to dump file

# File loading
print(f"File is loading: {plik_dump}...")
try:
    pipeline = import_file(plik_dump)
except Exception as e:
    print(f"Loading error, check file or filepath {e}")
    exit()
frame_max = pipeline.source.num_frames
print(f"File is loaded. Number of frames {pipeline.source.num_frames}")

File is loading: /home/przemek/Documents/temp/heating_pd_10k_10ps.dump...
File is loaded. Number of frames 301


## That part of the code calculates in loop CSPs and save figures into ./images/ directory

In [6]:
iterator = 1 # how often you want to calculate CSP

for frame_number in range (0, frame_max, iterator):
    #Modifier
    procent_done = (frame_number/frame_max)*100
    pipeline.modifiers.append(CentroSymmetryModifier())
    data = pipeline.compute(frame_number) # data is used to apply modifier for chosen frame
    csp = data.tables['csp-centrosymmetry'] # export data to table
    time = frame_number/100
    # plot export
    png_output_directory = "/home/przemek/Documents/temp/images_csp/"
    if not os.path.exists(png_output_directory):
        os.makedirs(png_output_directory)
    filename_png = os.path.join(png_output_directory, "csp_{}.png".format(frame_number))
    #print(csp.xy())
    csp_xy_data = csp.xy()
    x_values = csp_xy_data[:, 0]
    y_values = csp_xy_data[:, 1]
    plt.title(f"Time {time:.1f} ps") # plot title
    plt.plot(x_values, y_values)
    plt.xlim(0, 20)
    plt.ylim(0, 20000)
    plt.xlabel("CSP value")
    plt.ylabel("Counts")
    plt.title(f"CSP evolution with time {time:.2f} ps")
    plt.savefig(filename_png)
    plt.close()
    #print(f"Done {procent_done:.1f} %")
    pipeline.modifiers.clear()
#plt.show()
print(f"Done 100 %")

Done 100 %


## Here we create movie. If you want you can control fps and outfile name. Just run the code to make a movie.

In [12]:
import os
from moviepy import ImageSequenceClip

fps = 10
output_video_file_example = 'test_video.mp4'

def create_video_from_images(image_folder, output_video_file, fps, codec='libx265', bitrate=None): 
    # Checking of the .png files
    image_files = [img for img in os.listdir(image_folder) if img.endswith('.png')]

    # Sorting of files based on the file number - assuming template name_number.png
    def get_numeric_part(filename_with_ext):
        basename = os.path.basename(filename_with_ext)
        name_without_ext = os.path.splitext(basename)[0]
        try:
            numeric_string = name_without_ext.split('_')[-1] #convert string to int
            return int(numeric_string)
        except (IndexError, ValueError):
            print(f"I cannot divide name of the file: {filename_with_ext}")
            return -1 

    full_path_images = [os.path.join(image_folder, img) for img in image_files]
    full_path_images.sort(key=get_numeric_part)
    if not full_path_images:
        print("Error - there is not .png files in directory")
        return

    #print("Sorted files to export:", full_path_images) #can be commented

    # Make a movie
    clip = ImageSequenceClip(full_path_images, fps=fps)
    
    # Movie params
    if bitrate:
        clip.write_videofile(output_video_file, codec=codec, bitrate=bitrate)
    else:
        clip.write_videofile(output_video_file, codec=codec)

    print(f"Movie exported as {output_video_file}")

# Przykład użycia:
# Upewnij się, że masz obrazy w folderze 'images/' nazwane np. 'wykres_format_0.png', 'wykres_format_1.png', itd.
# np. tworząc przykładowe pliki:
image_folder_example = './Documents/temp/'
if not os.path.exists(image_folder_example):
    os.makedirs(image_folder_example)
# for i in range(5):
#    with open(os.path.join(image_folder_example, f"wykres_format_{i}.png"), "w") as f:
#        f.write("dummy png content") # To są tylko puste pliki dla testu struktury

output_video_file_example = 'CSP_WS_temp_time.mp4' #video file name
create_video_from_images(image_folder_example, output_video_file_example, fps=10, codec='libx264', bitrate='5000k')

[]
Error - there is not .png files in directory


In [ ]:
rdf = data.tables['coordination-rdf']
print(rdf.xy())